# Generic

This is mostly copied from the previous notebook

In [1]:
import math
import base64
import binascii
import numpy as np
import matplotlib.pyplot as plt

def to_bytes(d, format):
    if isinstance(d, (bytes, bytearray)):
        return d
    elif format == 'hex':
        return bytes(bytearray.fromhex(d))
    elif format == 'base64':
        return base64.b64decode(d)
    elif format == 'str' or format == 'bytes':
        return d.encode()
    elif format == 'int':
        return to_bytes(hex(d)[2:], 'hex')

def bytes_to(b, format):
    if not isinstance(b, (bytes, bytearray)):
        return b
    elif format == 'hex':
        return binascii.hexlify(b).decode()
    elif format == 'base64':
        return base64.b64encode(b).decode()
    elif format == 'str':
        return b.decode()
    elif format == 'bytes':
        return b
    elif format == 'int':
        return int(bytes_to(b, 'hex')[2:], 16)
    
def draw_bytestring(bs):
    """Represent a byte string as a matrix of color
    """
    tmp = bs + bytes(math.ceil(math.sqrt(len(bs)))**2 - len(bs))
    tmp = [int(b) for b in tmp]
    m = [[tmp[i:i+int(math.sqrt(len(tmp)))] for i in range(0, len(tmp), int(math.sqrt(len(tmp))))]]
    matrix = np.matrix(m[0])
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_aspect('equal')
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.binary)
    plt.colorbar()
    plt.show()

## Helpers

This is mostly copied from the previous notebook

In [2]:
import math
import itertools

def xor(d1, d2, format):
    b1,b2 = to_bytes(d1, format), to_bytes(d2, format)
    return bytes_to(bytes(char1 ^ char2 for char1,char2 in zip(b1,b2)), format)

def pad(text, blocksize=16):
    padsize = blocksize - (len(text) % blocksize)
    if padsize == 0:
        padsize = blocksize
    return text + bytes([padsize]*padsize)
assert pad(b"YELLOW SUBMARINE", blocksize=20) == b'YELLOW SUBMARINE\x04\x04\x04\x04'

def unpad(text):
    """Validate padding and return unpadded value"""
    assert text[-text[-1]:] == bytes([text[-1]]*text[-1])
    return text[:-text[-1]]
assert unpad(pad(b"this is random text", blocksize=7)) == b"this is random text"

#### Crypto

Generic crypto stuff

In [3]:
from Crypto.Cipher import AES

def decrypt_aes_ecb(cipher, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        return unpad(aes.decrypt(cipher))
    else:
        return aes.decrypt(cipher)
def encrypt_aes_ecb(clear, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        clear = pad(clear)
    return aes.encrypt(clear)
assert decrypt_aes_ecb(encrypt_aes_ecb(b'test', 'YELLOW SUBMARINE'), 'YELLOW SUBMARINE') == b'test'

def encrypt_aes_cbc(clear, key, IV=None):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = pad(clear)
    prev_block = IV
    cipher = b''
    for blockstart in range(0, len(clear), 16):
        block = clear[blockstart:blockstart+16]
        cipher_block = encrypt_aes_ecb(xor(prev_block, block, 'bytes'), key, use_padding=False)
        prev_block = cipher_block
        cipher += cipher_block
    return cipher

def decrypt_aes_cbc(cipher, key, IV=None):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = b''
    prev_block = IV
    for blockstart in range(0, len(cipher), 16):
        block = cipher[blockstart:blockstart+16]
        tmp = decrypt_aes_ecb(block, key, use_padding=False)
        clear_block = xor(prev_block, tmp, 'bytes')
        prev_block = cipher[blockstart:blockstart+16]
        clear += clear_block
    return unpad(clear)
assert decrypt_aes_cbc(encrypt_aes_cbc(b'test'*200, b'YELLOW SUBMARINE'), b'YELLOW SUBMARINE') == b'test'*200

def aes_ctr_generator(key, nonce=0):
    """Generate a keystream using AES-CTR.
    """
    if isinstance(nonce, bytes):
        nonce = int.from_bytes(nonce, 'big')
    while True:
        counter = nonce.to_bytes(9, 'big') + b'\x00'*7
        yield encrypt_aes_ecb(counter, key, use_padding=False)
        nonce += 1
        
def encrypt_aes_ctr(key, cleartext, nonce=0):
    """Encrypt some text with AES-CTR.
    """
    cipher = b''
    keystream = aes_ctr_generator(key, nonce=nonce)
    for i in range(0, len(cleartext), 16):
        block = cleartext[i:i+16]
        cipher += xor(block, next(keystream), 'bytes')
    return cipher

decrypt_aes_ctr = encrypt_aes_ctr # Such elegance !

# Cryptopals notebook - part 3

## Set 4

### Exercice 25

First, let's get the cleartext from the ECB exercice (exercice 7 - AES ub ECB mode).

In [4]:
with open('set1_7.txt') as f:
    s = to_bytes(f.read().replace('\n', '').strip(), 'base64')
    
cleartext = decrypt_aes_ecb(s, 'YELLOW SUBMARINE')

Then we reencrypt it with CTR and a random key.

In [5]:
import os
key = os.urandom(32)
cipher = encrypt_aes_ctr(key, cleartext)

In [6]:
# Implement the targeted change function
import math
def edit(ciphertext, key, offset, newtext):
    # Re-build the generator
    keystream = aes_ctr_generator(key)
    
    # Now, lets splice the cipher text with the new one
    splice_over = False
    new_cipher = b''
    i = 0
    while i < len(ciphertext):
        block = ciphertext[i:i+16]
        if i + 16 < offset or splice_over:
            # Unchanged block
            new_cipher += block
            next(keystream)
            i += 16
        else:
            # In splice territory !
            # First, get all the blocks in cleartext that we need
            new_offset = offset % 16
            blockcount = math.ceil((len(newtext)+new_offset) / 16)
            cleartext = b''
            keyblocks = []
            for bc in range(blockcount):
                block = ciphertext[i+bc*16:i+bc*16+16]
                keyblocks.append(next(keystream))
                cleartext += xor(block, keyblocks[-1], 'bytes')
            
            # New splice the new text into the cleartext
            new_cleartext = cleartext[:new_offset] + newtext + cleartext[new_offset+len(newtext):]
            
            # And reencrypt
            new_ciphertext = b''
            for bc in range(blockcount):
                block = new_cleartext[bc*16:bc*16+16]
                new_ciphertext += xor(block, keyblocks[bc], 'bytes')
                
            # And wrap up
            new_cipher += new_ciphertext
            i += blockcount*16
            splice_over = True
    return new_cipher

In [7]:
# Test the edit function
import os
test_key = os.urandom(32)
test_cipher = encrypt_aes_ctr(test_key, b"This is a test text ! Wow ! It is marvelous. Truly.")
test_edited = edit(test_cipher, test_key, 3, b"INSERT ME")
test_clear = decrypt_aes_ctr(key=test_key, cleartext=test_edited)
assert test_clear == b'ThiINSERT MEst text ! Wow ! It is marvelous. Truly.'

In [8]:
# Actual deciphering
controlled_cipher = edit(cipher, key, 0, b'\x00'*len(cipher))
deciphered_cleartext = xor(controlled_cipher, cipher, 'bytes')
print(deciphered_cleartext)
assert deciphered_cleartext == cleartext

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

### Exercice 26

First, copy the helpers from the CBC bit-flipping exercice (set 2 - Exercice 16) but change the CBC method to CTR.

In [9]:
def build_userdata(data):
    key = b',\xe8~mv\xca\xce\x1c \xd4\x83\x9a\x12\xd2\xd7"'
    d =  pad(b'comment1=cooking%20MCs;userdata=' + data.replace(b'=', b'').replace(b';', b'') + \
            b';comment2=%20like%20a%20pound%20of%20bacon')
    return encrypt_aes_ctr(key, d)

def is_admin(encrypted_data):
    key = b',\xe8~mv\xca\xce\x1c \xd4\x83\x9a\x12\xd2\xd7"'
    data = decrypt_aes_ctr(key, encrypted_data)
    print(data)
    return b';admin=true;' in data

assert not is_admin(build_userdata(b'test'))

b'comment1=cooking%20MCs;userdata=test;comment2=%20like%20a%20pound%20of%20bacon\x02\x02'


The inject it trivially

In [10]:
# Get a few bytes of keystream
lenprefix = len(b'comment1=cooking%20MCs;userdata=')
encrypted_userdata = build_userdata(b'\x00'*50)
keystream_data = encrypted_userdata[lenprefix:lenprefix+100]

# Now inject the desired data
injected_data = xor(b';admin=true;', keystream_data[:len(b';admin=true;')], 'bytes')
injected_userdata = encrypted_userdata[:lenprefix] + injected_data + encrypted_userdata[lenprefix+len(injected_data):]
assert is_admin(injected_userdata)

b'comment1=cooking%20MCs;userdata=;admin=true;\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00;comment2=%20like%20a%20pound%20of%20bacon\x04\x04\x04\x04'


### Exercice 27

First copy the code from the CBC exercice (set 2 - Exercice 16) and adapt them to use IV=Key

In [11]:
def encrypt_aes_cbc_mod(clear, key):
    """CBC using the ECB mode of above
    """
    IV=key
    clear = pad(clear)
    prev_block = IV
    cipher = b''
    for blockstart in range(0, len(clear), 16):
        block = clear[blockstart:blockstart+16]
        cipher_block = encrypt_aes_ecb(xor(prev_block, block, 'bytes'), key, use_padding=False)
        prev_block = cipher_block
        cipher += cipher_block
    return cipher

def decrypt_aes_cbc_mod(cipher, key):
    """CBC using the ECB mode of above
    """
    IV=key
    clear = b''
    prev_block = IV
    for blockstart in range(0, len(cipher), 16):
        block = cipher[blockstart:blockstart+16]
        tmp = decrypt_aes_ecb(block, key, use_padding=False)
        clear_block = xor(prev_block, tmp, 'bytes')
        prev_block = cipher[blockstart:blockstart+16]
        clear += clear_block
    try:
        clear = unpad(clear)
    except:
        pass # Meh.
    def find_high_ascii(x):
        return x > 127
    is_high_ascii_list = list(map(find_high_ascii, clear))
    return clear, any(is_high_ascii_list)
assertion_clear, assertion_high_ascii = decrypt_aes_cbc_mod(encrypt_aes_cbc_mod(b'test'*200, b'YELLOW SUBMARINE'), b'YELLOW SUBMARINE')
assert assertion_clear == b'test'*200
assert not assertion_high_ascii

In [12]:
# Sender has this :
key = b'\xd9\x89K\xc5\x8e\x0c\x15\xb3\xecn\xcen2\xc2\x1fr'
cleartext = b'YELLOW SUBMARINE1234567890123456YELLOW SUBMARINE'
ciphertext = encrypt_aes_cbc_mod(cleartext, key)

# Attacker intercepts message and modifies it
new_ciphertext = ciphertext[:16] + b'\x00'*16 + ciphertext[:16]

# Receiver tries to decode new ciphertext and fails
new_cleartext, is_high_ascii = decrypt_aes_cbc_mod(new_ciphertext, key)
if is_high_ascii:
    print('Oh noes, the cipher was incorrect ! A high ASCII caracter was found !')
    print('The decrypted cipher was :', new_cleartext)
    
# Attacker gets the error message and tries to get the key from it
new_key = xor(new_cleartext[:16], new_cleartext[32:], 'bytes')
assert new_key == key
print('Extracted key :', new_key)

Oh noes, the cipher was incorrect ! A high ASCII caracter was found !
The decrypted cipher was : b'YELLOW SUBMARINE\xf0\x0e\ng\xd9%\xa5I6\xcef+\x9a\xc9\xa0\xaa\x80\xcc\x07\x89\xc1[5\xe0\xb9,\x83/`\x8bQ7'
Extracted key : b'\xd9\x89K\xc5\x8e\x0c\x15\xb3\xecn\xcen2\xc2\x1fr'


### Exercice 28

SHA 1 implementation taken from [here](https://github.com/pcaro90/Python-SHA1/blob/master/SHA1.py), validated quickly agaisnt the value given by DDG. The modifications for the next exercice are directly contained in this implementation.

In [211]:
class SHA1:
    def __init__(self, _H=None, _len_state=None):
        self._len_state = _len_state
        if _H is None:
            self.__H = [
                0x67452301,
                0xEFCDAB89,
                0x98BADCFE,
                0x10325476,
                0xC3D2E1F0
                ]
        else:
            self.__H = _H

    def __str__(self):
        return ''.join((hex(h)[2:]).rjust(8, '0') for h in self.__H)

    # Private static methods used for internal operations.
    @staticmethod
    def __ROTL(n, x, w=32):
        return ((x << n) | (x >> w - n))

    @staticmethod
    def __padding(stream, _len_state=None):
        l = len(stream)  # Bytes
        if _len_state:
            l += _len_state
        hl = [int((hex(l*8)[2:]).rjust(16, '0')[i:i+2], 16)
              for i in range(0, 16, 2)]

        l0 = (56 - l) % 64
        if not l0:
            l0 = 64

        if isinstance(stream, str):
            stream += chr(0b10000000)
            stream += chr(0)*(l0-1)
            for a in hl:
                stream += chr(a)
        elif isinstance(stream, bytes):
            stream += bytes([0b10000000])
            stream += bytes(l0-1)
            stream += bytes(hl)

        return stream

    @staticmethod
    def __prepare(stream):
        M = []
        n_blocks = len(stream) // 64

        stream = bytearray(stream)

        for i in range(n_blocks):  # 64 Bytes per Block
            m = []

            for j in range(16):  # 16 Words per Block
                n = 0
                for k in range(4):  # 4 Bytes per Word
                    n <<= 8
                    n += stream[i*64 + j*4 + k]

                m.append(n)

            M.append(m[:])

        return M

    @staticmethod
    def __debug_print(t, a, b, c, d, e):
        print('t = {0} : \t'.format(t),
              (hex(a)[2:]).rjust(8, '0'),
              (hex(b)[2:]).rjust(8, '0'),
              (hex(c)[2:]).rjust(8, '0'),
              (hex(d)[2:]).rjust(8, '0'),
              (hex(e)[2:]).rjust(8, '0')
              )

    # Private instance methods used for internal operations.
    def __process_block(self, block):
        
        MASK = 2**32-1

        W = block[:]
        for t in range(16, 80):
            W.append(SHA1.__ROTL(1, (W[t-3] ^ W[t-8] ^ W[t-14] ^ W[t-16]))
                     & MASK)

        a, b, c, d, e = self.__H[:]

        for t in range(80):
            if t <= 19:
                K = 0x5a827999
                f = (b & c) ^ (~b & d)
            elif t <= 39:
                K = 0x6ed9eba1
                f = b ^ c ^ d
            elif t <= 59:
                K = 0x8f1bbcdc
                f = (b & c) ^ (b & d) ^ (c & d)
            else:
                K = 0xca62c1d6
                f = b ^ c ^ d

            T = ((SHA1.__ROTL(5, a) + f + e + K + W[t]) & MASK)
            e = d
            d = c
            c = SHA1.__ROTL(30, b) & MASK
            b = a
            a = T

            #SHA1.debug_print(t, a,b,c,d,e)

        self.__H[0] = (a + self.__H[0]) & MASK
        self.__H[1] = (b + self.__H[1]) & MASK
        self.__H[2] = (c + self.__H[2]) & MASK
        self.__H[3] = (d + self.__H[3]) & MASK
        self.__H[4] = (e + self.__H[4]) & MASK

    # Public methods for class use.
    def update(self, stream):
        stream = SHA1.__padding(stream, self._len_state)
        stream = SHA1.__prepare(stream)
        for block in stream:
            self.__process_block(block)

    def digest(self):
        return to_bytes(self.hexdigest(), 'hex')
    
    def digest_with_state(self):
        return self.digest(), self.__H

    def hexdigest(self):
        s = ''
        for h in self.__H:
            s += (hex(h)[2:]).rjust(8, '0')
        return s

In [212]:
sha1 = SHA1()
sha1.update(b'test')
assert sha1.hexdigest() == 'a94a8fe5ccb19ba61c4c0873d391e987982fbbd3'

In [213]:
def sha1mac(key, message):
    sha1 = SHA1()
    sha1.update(key + message)
    return sha1.digest_with_state()

In [214]:
# A few trivial verifications
message = b'Hello, world !'
key = b'foo'
mac = sha1mac(key, message)
assert sha1mac(key, b'Hello, warld !') != mac
assert sha1mac(b'bar', message) != mac

### Exercice 29

The modified SHA1 algorithm is defined in the previous exercice.
This is a completely valid SHA1 algorithm, with one twist : the internal state can be set to any value at initalisation. This includes the 5-integer array \_\_H and the length of the complete message.

We use this and the valid MAC that was given for the inconspicuous message the make a MAC length-extension attack, in order to generate a MAC that is valid for a message containing `admin=true`.

In [219]:
# We will try to forge a MAC for the following prefix
key = b'\xe6\xfa05\xf6\xd2\x05\x0fL,\x13,'
original_message = b"comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"
original_mac, sha1_state = sha1mac(key, original_message)
print("Original MAC", original_mac)
print("SHA1 State", sha1_state)


# In this example, we want to forge a MAC that has the same prefix as the original one.
# The attacker know the original message and the original MAC, but has no information about the key,
# except a vague idea about its length (let's say between 0 and 256 bytes).

# Validation function
def is_valid_mac(mac, message):
    tmp = sha1mac(key, message)[0]
    return tmp == mac
assert is_valid_mac(original_mac, original_message)

# The message we want to append
new_message_suffix = b";admin=true"

Original MAC b'b\xf8s\\\x14"\t\xee\xe4\xa5E\x91M\xec\x8e\xe3n\x12.['
SHA1 State [1660449628, 337775086, 3836036497, 1307348707, 1846685275]


In [220]:
# Bruteforce over keylength to get the desired result
for i in range(0, 255):

    # Apparently the sha1_state variable gets corrupted somewhere
    sha1_state = [1660449628, 337775086, 3836036497, 1307348707, 1846685275]

    glue_padding = sha1_padding(b'A'*i + original_message)[len(original_message)+i:]
    new_message = original_message + glue_padding + new_message_suffix

    # Let's generate the MAC, then find the message that corresponds to the MAC
    fake_sha1_generator = SHA1(_H=sha1_state, _len_state=i + len(original_message)+len(glue_padding))
    fake_sha1_generator.update(new_message_suffix)
    fake_mac = fake_sha1_generator.digest()

    if is_valid_mac(fake_mac, new_message):
        print("Found a valid MAC", fake_mac, "for keylength !", i)
        break

Found a valid MAC b'\r\xea\xcb\xef\xca\x92\xd8\x91a:N\xf1\xfe4M\xe7\x11z\xab\xa8' for keylength ! 12


### Exercice 30

MD4 length extension attack. Same idea as previously, other algorithm.

In [22]:
# Shamelessly copied from https://github.com/ricpacca/Cryptopals/blob/master/S4C30.py

from struct import pack, unpack

def left_rotate(value, shift):
    """Returns value left-rotated by shift bits. In other words, performs a circular shift to the left."""
    return ((value << shift) & 0xffffffff) | (value >> (32 - shift))

class MD4:
    """Adapted from: https://github.com/FiloSottile/crypto.py/blob/master/3/md4.py"""
    buf = [0x00] * 64

    _F = lambda self, x, y, z: ((x & y) | (~x & z))
    _G = lambda self, x, y, z: ((x & y) | (x & z) | (y & z))
    _H = lambda self, x, y, z: (x ^ y ^ z)

    def __init__(self, message, ml=None, A=0x67452301, B=0xefcdab89, C=0x98badcfe, D=0x10325476):
        self.A, self.B, self.C, self.D = A, B, C, D

        if ml is None:
            ml = len(message) * 8

        length = pack('<Q', ml)

        while len(message) > 64:
            self._handle(message[:64])
            message = message[64:]

        message += b'\x80'
        message += bytes((56 - len(message) % 64) % 64)
        message += length

        while len(message):
            self._handle(message[:64])
            message = message[64:]

    def _handle(self, chunk):
        X = list(unpack('<' + 'I' * 16, chunk))
        A, B, C, D = self.A, self.B, self.C, self.D

        for i in range(16):
            k = i
            if i % 4 == 0:
                A = left_rotate((A + self._F(B, C, D) + X[k]) & 0xffffffff, 3)
            elif i % 4 == 1:
                D = left_rotate((D + self._F(A, B, C) + X[k]) & 0xffffffff, 7)
            elif i % 4 == 2:
                C = left_rotate((C + self._F(D, A, B) + X[k]) & 0xffffffff, 11)
            elif i % 4 == 3:
                B = left_rotate((B + self._F(C, D, A) + X[k]) & 0xffffffff, 19)

        for i in range(16):
            k = (i // 4) + (i % 4) * 4
            if i % 4 == 0:
                A = left_rotate((A + self._G(B, C, D) + X[k] + 0x5a827999) & 0xffffffff, 3)
            elif i % 4 == 1:
                D = left_rotate((D + self._G(A, B, C) + X[k] + 0x5a827999) & 0xffffffff, 5)
            elif i % 4 == 2:
                C = left_rotate((C + self._G(D, A, B) + X[k] + 0x5a827999) & 0xffffffff, 9)
            elif i % 4 == 3:
                B = left_rotate((B + self._G(C, D, A) + X[k] + 0x5a827999) & 0xffffffff, 13)

        order = [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15]
        for i in range(16):
            k = order[i]
            if i % 4 == 0:
                A = left_rotate((A + self._H(B, C, D) + X[k] + 0x6ed9eba1) & 0xffffffff, 3)
            elif i % 4 == 1:
                D = left_rotate((D + self._H(A, B, C) + X[k] + 0x6ed9eba1) & 0xffffffff, 9)
            elif i % 4 == 2:
                C = left_rotate((C + self._H(D, A, B) + X[k] + 0x6ed9eba1) & 0xffffffff, 11)
            elif i % 4 == 3:
                B = left_rotate((B + self._H(C, D, A) + X[k] + 0x6ed9eba1) & 0xffffffff, 15)

        self.A = (self.A + A) & 0xffffffff
        self.B = (self.B + B) & 0xffffffff
        self.C = (self.C + C) & 0xffffffff
        self.D = (self.D + D) & 0xffffffff

    def digest(self):
        return pack('<4I', self.A, self.B, self.C, self.D)

    def hex_digest(self):
        return hexlify(self.digest()).decode()
    
    
def md_pad(message):
    """Pads the given message the same way the pre-processing of the MD4 algorithm does."""
    ml = len(message) * 8

    message += b'\x80'
    message += bytes((56 - len(message) % 64) % 64)
    message += pack('<Q', ml)

    return message

def state_from_digest(md4hash):
    # Unpack a md4 hash into the 4 state variables
    return unpack("<4I", b'\xdb4mi\x1dz\xccM\xc2b]\xb1\x9f\x9e?R')

In [23]:
import os
class MD4Oracle:
    
    def __init__(self):
        # Generate key
        self._key = os.urandom(12)
        
    def generate(self, message):
        # Generate a valid MAC
        return MD4(self._key + message).digest()
    
    def check(self, mac, message):
        # Check if a MAC is valid
        real_mac = self.generate(message)
        return real_mac == mac

In [24]:
# We will try to forge a MAC for the following prefix
oracle = MD4Oracle()
original_message = b"comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"
original_mac = oracle.generate(original_message)
md4_state = unpack('<4I', original_mac)
print("Original MAC", original_mac)
print("MD4 State", md4_state)
assert oracle.check(original_mac, original_message)

# In this example, we want to forge a MAC that has the same prefix as the original one.
# The attacker know the original message and the original MAC, but has no information about the key,
# except a vague idea about its length (let's say between 0 and 256 bytes).

# The message we want to append
new_message_suffix = b";admin=true"

Original MAC b'\x8939\x0e7\xcd\x93j\xcdD(\x8aV\xab\x99\xb8'
MD4 State (238629769, 1788071223, 2317894861, 3097078614)


In [25]:
# Bruteforce over keylength to get the desired result
for i in range(0, 15):

    glue_padding = md_pad(b'A'*i + original_message)[len(original_message)+i:]
    new_message = md_pad(b'A'*i + original_message)[i:] + new_message_suffix

    # Let's generate the MAC, then find the message that corresponds to the MAC
    fake_md4_generator = MD4(new_message_suffix, ml=(i + len(new_message))*8,
                             A=md4_state[0], B=md4_state[1], C=md4_state[2], D=md4_state[3])
    fake_mac = fake_md4_generator.digest()

    if oracle.check(fake_mac, new_message):
        print("Found a valid MAC", fake_mac, "for keylength !", i)
        break

Found a valid MAC b'\xe3\xcb\r\xdc\tL\x17\xff\x04\x81`OP\xe0\x0c\x9f' for keylength ! 12


### Exercice 31


Breaking HMAC-SHA1 timing leak. Shouldn't be too hard. Webserver and HMAC function are copied over here but should be called from another file to be able to run independently.

In [ ]:
# see exercice31.py file

In [11]:
# Attacker payload
import os
content = bytes_to(os.urandom(50), 'hex')

In [33]:
# Now, bruteforce the HMAC byte per byte. Simple.
import time, requests


current_hmac = b'\x00' * 20

def index_of_max(a):
    # Find the index of the highest value in an array as well as its index
    assert  len(a) > 0
    mx, idx = a[0], 0
    for i in range(1, len(a)):
        if mx < a[i]:
            mx, idx =  a[i], i
    return mx, idx

# A HMAC is 20 bytes long
for i in range(20):
    
    # And each byte may be between 0 and 255
    timers = []
    for b in range(255):

        current_hmac = current_hmac[:i] + bytes([b]) + current_hmac[i+1:]
        start = time.monotonic()
        requests.get("http://localhost:9999", params={"signature": bytes_to(current_hmac, 'hex'),
                                               "content": content})
        timers.append(time.monotonic() - start)
    
    # Find the byte that had the slowest answer. This one should be it.
    mx, idx = index_of_max(timers)
    current_hmac = current_hmac[:i] + bytes([idx]) + current_hmac[i+1:]
    print('Current key is', current_hmac)

    
print('\n\nKey should be', current_hmac)
req =  requests.get("http://localhost:9999", params={"signature": bytes_to(current_hmac, 'hex'),
                                              "content": content})
print('Is this the right key ?', req.json())

Current key is b'\xd3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16h\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16hw\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16hw\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16hw\x04,\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16hw\x04,\xbb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16hw\x04,\xbb+\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16hw\x04,\xbb+\xcc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16hw\x04,\xbb+\xcc\x93\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Current key is b'\xd3\x16hw\x04,\xbb+\xcc\x93H\x00\x00\x00\x00\x00\x00\x00\x00\

AttributeError: 'Response' object has no attribute 'body'

In [34]:
print('\n\nKey should be', current_hmac)
req =  requests.get("http://localhost:9999", params={"signature": bytes_to(current_hmac, 'hex'),
                                              "content": content})
print('Q', req.json())



Key should be b'\xd3\x16hw\x04,\xbb+\xcc\x93H\xd1\x9d\x9ev\xaf\xfb\xfe}\x82'
Q True


### Exercice 32

Basically the same but with lower artificial sleep.
Use following techniques :
- guess bytes in groups to amplify the early abort
- use statistical methods and a lot of retries to find slowest guess